## Requisitos

Es necesario instalar el ejecutable ffmpeg para poder ejecutar este script. Seguir los pasos de instalación del siguiente repo sobre la <a href='https://github.com/jiaaro/pydub?tab=readme-ov-file#installation'>libreria pydub</a>.

EN caso de recibir error al ejecutar la función Play(), sobre el fichero .py de play modificar la linea 14 de código sustituyendo por lo siguiente:
```python
with NamedTemporaryFile("w+b", suffix=".wav", delete=False) as f:
```

Para poder ejecutar la librería pyaudio es necesario instalar **portaudio** a nivel del equipo, esto solo es necesario para ejecutar la última celda en la que conectaremos el asistente al micrófono de nuestro equipo.

## Instlación de librerias

In [2]:
!pip install speechrecognition
!pip install ibm_watson_machine_learning
!pip install langchain
!pip install load_dotenv
!pip install ipython
!pip install ipykernel
!pip install gtts
!pip install pydub
!pip install langchain-community
!pip install pyaudio
!pip install wave

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 4.3 MB/s eta 0:00:0000:0100:01
  Using cached ipykernel-6.29.4-py3-none-any.whl.metadata (6.3 kB)
  Using cached appnope-0.1.4-py2.py3-none-any.whl.metadata (908 bytes)
  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached debugpy-1.8.1-cp312-cp312-macosx_11_0_universal2.whl.metadata (1.1 kB)
  Using cached jupyter_client-8.6.2-py3-none-any.whl.metadata (8.3 kB)
  Using cached jupyter_core-5.7.2-py3-none-any.whl.metadata (3.4 kB)
  Using cached nest_asyncio-1.6.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached psutil-5.9.8-cp38-abi3-macosx_11_0_arm64.whl.metadata (21 kB)
  Using cached pyzmq-26.0.3-cp312-cp312-macosx_10_15_universal2.whl.metadata (6.1 kB)
  Using cached platformdirs-4.2.2-py3-none-any.whl.metadata (11 kB)
Using cached ipykernel-6.29.4-py3-none-any.whl (117 kB)
Using cached comm-0.2.2-py3-none-any.whl (7.2 kB)
Using cached debugpy-1.8.1-cp312-cp312-macosx_11_0_universal2.whl (1.4 MB)
Using 

## Carga de Librerias

In [1]:
import speech_recognition as sr
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play

import pyaudio
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM


/Users/ignacioarriola/Documents/Global/Formacion/formacion_generative_ai/watsonx-env/lib/python3.12/site-packages/ibm_watson_machine_learning/foundation_models/extensions/langchain/llm.py:60: WatsonxLLMDeprecationWarning: ibm_watson_machine_learning.foundation_models.extensions.langchain.WatsonxLLM is deprecated and will not be supported in the future. Please import from langchain-ibm instead.
To install langchain-ibm run `pip install -U langchain-ibm`.
  _raise_watsonxllm_deprecation_warning()


## Definición de variables de entorno

In [2]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv('../.env')

try:
    REGION = os.environ["RUNTIME_ENV_REGION"]
except KeyError:
    # Set your region here if you are not running this notebook in the watsonx.ai Jupyter environment
    # us-south, eu-de, etc.
    REGION = "us-south"

try:
    api_key = os.environ["api_key"]
except KeyError:
    # Enter api key here if not running this notebook in the watsonx.ai Jupyter environment
    api_key = getpass.getpass("Please enter your WML api key (hit enter): ")

credentials = {
    "url": "https://" + REGION + ".ml.cloud.ibm.com",
    "apikey": api_key
}


In [3]:
import os

try:
    project_id = os.environ["project_id"]
except KeyError:
    # Enter project ID here if not running this notebook in the watsonx.ai Jupyter environment
    project_id = getpass.getpass("Please enter your WML project_id (hit enter): ")


Procesamos el audio y con el modelo de reconocimiento de voz de google transformamos el audio a texto.

In [4]:
def convert_audio_to_wav(ogg_file, wav_file, format='ogg'):
    audio = AudioSegment.from_file(ogg_file, format=format)
    audio.export(wav_file, format="wav")

def audio_to_text(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data, language='es-ES')
            return text
        except sr.UnknownValueError:
            return "No se pudo entender el audio"
        except sr.RequestError as e:
            return f"Error de solicitud; {e}"


Definimos el LLM para generar una respuesta al audio, y lo transformamos a formato audio con la libreria de transformación de texto a speech de google.

In [5]:
def get_model(model_id, credentials, project_id):
    model_id_1 = model_id
    parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE,
    GenParams.MAX_NEW_TOKENS: 50,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1,
    GenParams.STOP_SEQUENCES: ["."]
}
    model = Model(
        model_id=model_id_1,  
        params=parameters, 
        credentials=credentials,
        project_id=project_id)
    
    llm = WatsonxLLM(model=model)
    return llm

def generate_response(model, prompt):
    response = model.invoke(prompt)
    return response

def text_to_audio(text, output_file):
    tts = gTTS(text=text, lang='es')
    tts.save(output_file)


llm = get_model(ModelTypes.MIXTRAL_8X7B_INSTRUCT_V01_Q, credentials, project_id)

/Users/ignacioarriola/Documents/Global/Formacion/formacion_generative_ai/watsonx-env/lib/python3.12/site-packages/ibm_watson_machine_learning/foundation_models/utils/utils.py:290: LifecycleWarning: Model 'ibm-mistralai/mixtral-8x7b-instruct-v01-q' is in deprecated and constricted state from 2024-04-19 until 2024-06-20. IDs of alternative models: ibm-mistralai/mixtral-8x7b-instruct-v01. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(default_warning_template.format(


Combinamos todas las funciones previas y mostramos el audio generado.

In [6]:
def process_audio(origin_file, wav_file, llm, response_audio_file):
    convert_audio_to_wav(origin_file, wav_file, format='ogg')
    text = audio_to_text(wav_file)
    print(f"Texto reconocido: {text}")
    
    response = generate_response(llm, text)
    print(f"Respuesta generada: {response}")
    
    text_to_audio(response, response_audio_file)
    response_audio = AudioSegment.from_mp3(response_audio_file)
    
    play(response_audio)

In [16]:
# Nombre del archivo OGG de entrada y el archivo MP3 de salida
origin_file = "data/audio.ogg"
wav_file = "data/audio.wav"
response_audio_file = "data/response.mp3"

# Procesar el audio
process_audio(origin_file=origin_file, wav_file=wav_file, llm=llm, response_audio_file=response_audio_file)

Texto reconocido: qué es la Inteligencia Artificial generativa
Respuesta generada: ?

La Inteligencia Artificial generativa (IA generativa) es una rama de la Inteligencia Artificial que se enfoca en la creación de contenido nuevo y original, en lugar de simplemente


Input #0, wav, from '/var/folders/cb/jq2ptwjx1514d2n2fx3sd55m0000gn/T/tmpbw_zgof4.wav':
  Duration: 00:00:14.76, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s


En la siguiente celda definiremos una función, prácticamente igual que process_audio pero sin convertir de off a wav, que utilizaremos para preguntar al asistente desde el micrófono directamente.

In [12]:
def answer_question( wav_file, llm, response_audio_file):
    text = audio_to_text(wav_file)
    print(f"Texto reconocido: {text}")
    
    response = generate_response(llm, text)
    print(f"Respuesta generada: {response}")
    
    text_to_audio(response, response_audio_file)
    response_audio = AudioSegment.from_mp3(response_audio_file)
    
    play(response_audio)

## Asistente con microfono

En la siguiente celda incorporaremos las anteriores funcionalidades a la captura del audio por micrófono, para interactuar con el asistente por voz.

In [13]:
import pyaudio
import wave

CHUNK = 1024  # Tamaño de cada fragmento de audio
FORMAT = pyaudio.paInt16  # Formato de audio (16 bits por muestra)
CHANNELS = 1  # Número de canales (mono)
RATE = 44100  # Frecuencia de muestreo (44.1 kHz)

start = AudioSegment.from_wav("./data/start.wav")  # Cargar el archivo de audio "start.wav"

# Reproducir el archivo de inicio de mensaje
play(start)

p = pyaudio.PyAudio()  # Inicializar PyAudio

# Abrir el flujo de audio para grabación
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

frames = []  # Lista para almacenar los datos de audio grabados
seconds = 3  # Duración de la grabación en segundos

# Grabar el audio en fragmentos y almacenarlo en la lista "frames"
for i in range(0, int(RATE / CHUNK * seconds)):
    data = stream.read(CHUNK)
    frames.append(data)

# Detener y cerrar el flujo de audio
stream.stop_stream()
stream.close()
p.terminate

stop = AudioSegment.from_wav("./data/stop.wav")  # Cargar el archivo de audio "stop.wav"
play(stop)  # Reproducir el archivo fin de mensaje
print("recording stopped")  # Imprimir mensaje indicando que la grabación ha terminado

# Guardar los datos de audio grabados en un archivo WAV
wf = wave.open("./data/question_temp.wav", 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

response_audio_file = "data/response.mp3"  # Ruta del archivo de audio de respuesta

# Definir una función para procesar el audio
def process_audio2(wav_file, llm, response_audio_file):
    text = audio_to_text(wav_file)  # Convertir el audio a texto
    print(f"Texto reconocido: {text}")  # Imprimir el texto reconocido
    
    response = generate_response(llm, text)  # Generar una respuesta usando un modelo de lenguaje (LLM)
    print(f"Respuesta generada: {response}")  # Imprimir la respuesta generada
    
    text_to_audio(response, response_audio_file)  # Convertir la respuesta de texto a audio
    response_audio = AudioSegment.from_mp3(response_audio_file)  # Cargar el archivo de audio de respuesta
    
    play(response_audio)  # Reproducir el archivo de audio de respuesta

# Llamar a la función para procesar el audio grabado y generar una respuesta
answer_question(wav_file="./data/question_temp.wav", llm=llm, response_audio_file=response_audio_file)

os.remove("./data/question_temp.wav") # Eliminar el WAV de la pregunta


recording stopped
Texto reconocido: qué es la inteligencia artificial
Respuesta generada:  y cómo funciona

La inteligencia artificial (IA) es una rama de la ciencia de la computación que trata de desarrollar sistemas informáticos que puedan realizar tareas que requieren
